In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory to the path so we can import the module
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
from modules.needleman_wunsch.nw_core import *
from modules.result_evaluation import *
from modules.carillo_lipman_bounds import *
from modules.needleman_wunsch.nw_cost import *

In [ ]:
	
from Bio import AlignIO
from Bio import pairwise2
from Bio.Align import MultipleSeqAlignment
from Bio import SeqIO

# Needleman-Wunsch Algorithm Implementation

This project implements multiple versions of the Needleman-Wunsch algorithm:

- **Pairwise sequence alignment** (classic Needleman-Wunsch).  
- **N-sequence progressive alignment**, where a list of \( N \) sequences is aligned sequentially:  
  - First, sequences \( S_0 \) and \( S_1 \) are aligned.  
  - Then, sequence \( S_2 \) is aligned to the existing alignment of \( S_0 \) and \( S_1 \).  
  - This process continues iteratively until all \( N \) sequences are aligned.    
- **N+M sequence alignment**, where two pre-aligned groups of \( N \) and \( M \) sequences are merged.  
- **Multidimensional algorithm** for simultaneous alignment of K sequences.

In all implementations, **gap opening cost > gap extension cost** to encourage the formation of fewer, larger gaps rather than multiple small gaps. This approach better reflects biological reality, as insertions and deletions in DNA, RNA, and protein evolution tend to occur in longer segments rather than isolated positions.

The functions support alignment of both **nucleotide** (DNA/RNA) and **protein** sequences:

- **For DNA/RNA alignment**, set `blosum_m=False`. The scoring scheme is:  
  - **Match** = \( +1 \)  
  - **Mismatch** = \( -1 \)  
  - **Gap opening** = \( -10 \)  
  - **Gap extension** = \( -2 \)  

- **For protein alignment**, set `blosum_m=True` to use the **BLOSUM62** substitution matrix.

Additionally, visualization functions are provided for **alignment matrices, final sequence alignment**, and other outputs to enhance clarity and debugging.

#### DNA/RNA examples

The simpliest short example.

In [ ]:
blossum = False
sequences = ["ATCG", "AG"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=True)

Let's try longer sequences.

In [ ]:
blossum = False
sequences = ["ATCGTACGTCCTAGGCTAAGCTTAGCGTACGATCGTTAGCTA", "ATGCCGTTAGCCTAGGCTAAGCGTACGATCGTAGCTATTTA"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=False)
print_alignments(alignment)
print("Score: ", score)

Now we will test Needleman-Wunsch algorithm for N-sequence progressive alignment.

In [ ]:
blossum = False
sequences = ["TAGCCT", "CCATGCT", "TAGCCCTA", "CATGCT", "AGCT", "TAGTA"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=True)
print_alignments(alignment)
print("Score: ", score)

Finally, let's test Needleman-Wunsch algorithm for **N+M sequence alignment**, where two pre-aligned groups of \( N \) and \( M \) sequences are merged.  

In [ ]:
blossum = False

sequences = ["TAGCCT", "CCATGCT"]
_, sequences1 = needleman_wunsch(sequences, blossum, print_result=False)
print("Aligned sequences 1: ", sequences1)

sequences = ["CATGCT", "CAGCT"]
_, sequences2 = needleman_wunsch(sequences, blossum, print_result=False)
print("Aligned sequences 2: ", sequences2)

score, alignment = needleman_wunsch_multiple(sequences1, sequences2, blossum, print_result=True)

#### Protein examples

To test it we need just change blossum=True.

In [ ]:
sequences = ["CHAT", "CAT"]

score, alignment = needleman_wunsch(sequences, True, print_result=True)

In [ ]:

blossum = True
sequences = ["CHAT", "CAT", "HER", "HAT", "HARAT"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=True)

In [ ]:

blossum = True
sequences = ["CANSAT", "CANS", "CAN"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=True)

In [ ]:
blossum = True
sequences1 = ["CHAT", "C-AT"]
sequences2 = ["HER--", "HA--T", "HARAT"]

score, alignment = needleman_wunsch_multiple(sequences1, sequences2, blossum, print_result=True)

## Evaluation of the Needleman-Wunsch Algorithm

To verify the correctness of the classic Needleman-Wunsch algorithm, I compared my implementation with the reference implementation from the **Biopython** library. 

For evaluating the alignment quality, the **sum-of-pairs (SP) score** was used. This metric is widely used in multiple sequence alignment because it provides a straightforward way to assess alignment quality by summing the pairwise similarity scores across all aligned positions. It effectively captures the consistency of alignments and allows direct comparison between different methods.

Additionally, the implemented **alignment comparison function** accounts for cases where sequences in the two alignments may be presented in a different order. It calculates the **SP-score for both possible orderings** and selects the maximum value, ensuring that identical alignments are correctly recognized as fully matching, even if the sequence order differs.

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq1 = "CHAT"
seq2 = "CAT"

test_biopython_vs_custom(seq1, seq2, True, gap_open_penalty, gap_extension_penalty)

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq1 = "MCGNIQLEYAHHGPATQFLWTYIMIGCLKFKGFREQHFYIPGICKDWHFKFLCFYRMIHIPIGPGYITQNTSPAGHYRHSEKAICVMQMFKYICRFRA"
seq2 = "MHGQLEYIAHSPATRFLYTIGCLKFKWFREHHFNIPGECKDWHFKFDCFYRMIHIPIGPAIMYITSPAGHYRHSEMAITVMQMNKVGCRFRDICLYFVES"

test_biopython_vs_custom(seq1, seq2, True, gap_open_penalty, gap_extension_penalty)

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq1 = "MCGNIQLEYAHHGPATQFLWTYIMIGCLKFKGFRQHFYIPGICKDWHFKFLCFYRMIHIPIYITQNTSPAGHYRHSEKAICVMQMFKYICRFRA"
seq2 = "MHGQLEYIAHSPATRFLYTIGCLKFKWFRIPGECKDWHFKFDCFYRMIHIPIGPAIMYITSPAGHYRHSEMAITVMQMNKVGCRFRDICLYFVES"

test_biopython_vs_custom(seq1, seq2, True, gap_open_penalty, gap_extension_penalty)

# Multidimensional Extension of the Needleman-Wunsch Algorithm

This project extends the classic Needleman-Wunsch algorithm to support **simultaneous multiple sequence alignment** by constructing a **multidimensional dynamic programming (DP) matrix**. Unlike the progressive approach, where sequences are aligned sequentially, this method builds an \( N^K \) matrix directly, considering all \( K \) sequences at once.

### **Scoring Scheme**
The algorithm supports alignment of both **nucleotide (DNA/RNA)** and **protein sequences**:
- **For DNA/RNA alignment**, set `blosum_m=False`. The scoring scheme is:  
  - **Match** = \( +1 \)  
  - **Mismatch** = \( -1 \)  
  - **Gap opening** = \( -10 \)  
  - **Gap extension** = \( -2 \)  

- **For protein alignment**, set `blosum_m=True` to use the **BLOSUM62** substitution matrix.

Additionally, the cost function distinguishes between **gap opening** and **gap extension**, ensuring 

biologically realistic alignments by favoring longer gaps over multiple small insertions.

### **Matrix Visualization**
A key feature of this implementation is the ability to **visualize matrix slices** when aligning exactly **three sequences** (\( K=3 \)). This allows for:
- **Heatmap visualization** of DP matrix values for each slice along the third dimension.
- **Comparison of alignment scores** across different prefixes.
- **Identification of alignment trends and patterns** in a more intuitive way.

However, for \( K > 3 \), direct visualization becomes impractical due to dimensionality constraints. Only **2D slices of the 3D matrix** can be visualized, meaning **traceback arrows are not displayed** in the plots.

This extension significantly enhances the ability to perform **exact multiple sequence alignment** while also providing useful visualization tools for debugging and analysis.

In [ ]:
sequences = ["HER", "CAT", "HAT"]
blosum_m = True
carillo = False
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(
    sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, 
    print_result=True, identity_score=1, substitution_score=-1
    )

In [ ]:
sequences = ["CHAT", "RAT", "CAT"]
blosum_m = True
carillo = False
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(
    sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, 
    print_result=True, identity_score=1, substitution_score=-1
    )

In [ ]:
sequences = ["HER", "CAT", "HAT", "RAT", "CHAT", "HARAT"]
blosum_m = True
carillo = False
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=True, identity_score=1, substitution_score=-1)

# Carillo-Lipman Bounds in Multiple Sequence Alignment
## Introduction

The **Carillo-Lipman bounding method** is an optimization technique used in **multiple sequence alignment (MSA)** to reduce the computational complexity of aligning multiple biological sequences. This method allows for a more efficient dynamic programming approach by identifying **bounds** on where each sequence can be placed within an alignment.

Instead of constructing a full **N-dimensional Needleman-Wunsch matrix**, the Carillo-Lipman approach utilizes **pairwise alignments** to determine **upper and lower bounds** for the placement of each sequence. These bounds limit the search space, reducing unnecessary computations and improving efficiency.

## Using Carillo-Lipman Bounds in Multidimensional Needleman-Wunsch

The Carillo-Lipman bounds help **reduce the number of computed cells** in the multidimensional Needleman-Wunsch algorithm by defining **valid alignment regions**. The main steps are:

1. **Compute Carillo-Lipman bounds for each sequence**:
   - Define the **min/max valid indices** for each sequence.
   - Precompute the **maximum possible gaps** before the last letter.

2. **Initialize the DP matrix**:
   - Set up the matrix **only within valid bounds**.
   - Track the remaining **available gaps** for each sequence.

3. **Fill the DP matrix efficiently**:
   - Iterate over **only valid indices** within Carillo-Lipman constraints.
   - Before computing the score:
     - Check if **remaining gaps** would become negative.
     - If so, **skip the transition**.
   - If **no valid transitions exist** → assign **\( -\infty \)** to the cell.

4. **Traceback to reconstruct the alignment**.

## Carillo-Lipman Efficiency Calculation

The efficiency of Carillo-Lipman bounds in the **Multidimensional Needleman-Wunsch algorithm** is calculated as:

$$
\text{efficiency} = \frac{\text{skipped cells}}{\text{total possible cells}}
$$

Where:
- **Skipped cells** — number of matrix cells that were skipped due to Carillo-Lipman bounds.
- **Total possible cells** — the total number of cells in the dynamic programming matrix.

In [ ]:
sequences = ["HER", "CAT", "HAT", "RAT", "CHAT", "HARAT"]
blosum_m = True
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

compute_carillo_lipman_bounds(
    sequences, blosum_m, True, gap_opening_score, gap_extension_score, 
    identity_score=1, substitution_score=-1)

In [ ]:
sequences = ["CANSAT", "CAN", "CANS"]
blosum_m = True
carillo = False
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=True, identity_score=1, substitution_score=-1)

In [ ]:
sequences = ["CANSAT", "CAN", "CANS"]
blosum_m = True
carillo = True
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=True, identity_score=1, substitution_score=-1)

In [ ]:
seq1 = "MHGQLEYEAHHGPAT"
seq2 = "MHGQLEYAHSPATRFLYTIG"
seq3 = "MHGQLEYIPATRFL"
sequences = [seq1, seq2, seq3]
blosum_m = True
carillo = True
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=True, identity_score=1, substitution_score=-1)

In [ ]:
seq1 = "MHGQLEYEAHHGPAT"
seq2 = "MHGQLEYAHSPATRFLYTIG"
seq3 = "MHGQLEYIPATRFL"
sequences = [seq1, seq2, seq3]
blosum_m = True
carillo = False
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

needleman_wunsch_multidim(sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=True, identity_score=1, substitution_score=-1)

# Carillo-Lipman Bounds in Multiple Sequence Alignment: real data test

To evaluate the performance of my **multidimensional Needleman-Wunsch algorithm**, **BAliBASE** was used, a widely recognized benchmark for multiple sequence alignment (MSA). 

**BAliBASE (Benchmark Alignment dataBASE)** provides high-quality **reference alignments** for testing and validating MSA algorithms. It contains **manually curated** alignments based on structural and evolutionary data, making it an excellent benchmark for evaluating alignment accuracy.

For assessing alignment quality, the **Sum-of-Pairs (SP) metric** was used, which is one of the most commonly used scoring methods in MSA. The SP metric calculates the **pairwise alignment score across all sequence pairs**, providing an **objective measure of how well sequences are aligned** compared to the reference. 

By combining **BAliBASE reference alignments** with the **SP metric**, I can effectively **test and validate** the performance of my algorithm, ensuring its accuracy and robustness.

In [ ]:
unaligned_file = "../balibase/RV11.unaligned/BB11001.fasta"
aligned_file = "../balibase/RV11.aligned/BB11001.fasta"

unaligned_sequences = [str(record.seq) for record in SeqIO.parse(unaligned_file, "fasta")]
reference_alignment = [str(record.seq) for record in SeqIO.parse(aligned_file, "fasta")]

print("Unaligned sequences: ")
for seq in unaligned_sequences[:-1]:
    print(seq)

In [ ]:
sequences = unaligned_sequences[:-1]
blosum_m = True
carillo = True
gap_opening_score = -10
gap_extension_score = -2
identity_score = 1
substitution_score = -1

score, aligned_sequences = needleman_wunsch_multidim(
    sequences, blosum_m, carillo, gap_opening_score=-10, gap_extension_score=-2, print_result=False, identity_score=1, substitution_score=-1)

In [ ]:
print("Custom alignment: ")
print_alignments(aligned_sequences)
print()
print("Reference alignment: ")
print_alignments(reference_alignment)

Too long for real 4 sequences alignment even with Carillo-Lipman method. With alignments only for 3 sequences difficult to compare, because in case of 4 sequences we will get another alignment than for 3 sequences.